In [1]:
import header

import numpy as np
import pandas as pd
import spacy

import warnings
import time
import pickle
import os
import nltk

from nltk.tokenize import word_tokenize
from joblib import dump, load
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

In [17]:
fast_coeff = 10

X_train = pd.read_csv(
        "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/X_train_update.csv",
        sep=',',
        usecols=lambda column: column not in [
            'Unnamed: 0',
            'imageid',
            'description'
        ]
    )
X_test_challenge = pd.read_csv(
    "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/X_test_update.csv",
    sep=',',
    usecols=lambda column: column not in [
        'Unnamed: 0',
        'imageid',
        'description'
    ]
)
Y_train = pd.read_csv(
    "/Users/welto/Library/CloudStorage/OneDrive-CentraleSupelec/2A/CASA/RakutenPjct/data/Y_train_CVw08PX.csv",
    sep=',',
    usecols=lambda column: column != 'Unnamed: 0'
)

X_train = X_train['designation'][:X_train.shape[0]//fast_coeff].tolist()
X_test_challenge = X_test_challenge['designation'][:X_test_challenge.shape[0]//fast_coeff].tolist()

Y_train = Y_train['prdtypecode'][:Y_train.shape[0]//fast_coeff].tolist()

In [25]:
nltk.download("punkt")

X_train_clean = []
X_test_challenge_clean = []

a = len(X_train)
b = len(X_test_challenge)

header.progress_bar(
    0,
    a,
    prefix='Progress:',
    suffix='Complete',
    length=50
)

preprocessing_start_time = time.time()

for k in range(a):
    tokens = word_tokenize(
        header.normalize_accent(X_train[k].lower()),
        language='french'
    )
    X_train_clean.append(tokens)
    header.progress_bar(
        k + 1,
        a,
        prefix='X_train_raw_designation_clean :',
        suffix='Complete',
        length=50
    )

for k in range(b):
    tokens = word_tokenize(
        header.normalize_accent(X_test_challenge[k].lower()),
        language='french'
    )
    X_test_challenge_clean.append(tokens)
    header.progress_bar(
        k + 1,
        b,
        prefix='X_test_raw_designation_clean :',
        suffix='Complete',
        length=50
    )

X_train_strings = [' '.join(tokens) for tokens in X_train_clean]
X_test_strings = [' '.join(tokens) for tokens in X_test_challenge_clean]

preprocessing_end_time = time.time()
preprocessing_time_h, preprocessing_time_min, preprocessing_time_s = header.convert_seconds(
    preprocessing_end_time - preprocessing_start_time)

print(f"Preprocessed in {int(preprocessing_time_h)}h {int(preprocessing_time_min)}min {int(preprocessing_time_s)}s")

X_train_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete
X_test_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete


[nltk_data] Downloading package punkt to /Users/welto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: 'str' object has no attribute 'is_punct'

In [ ]:
X_train_strings

In [29]:
import nltk
from nltk.tokenize import word_tokenize
import string
import time

nltk.download("punkt")

X_train_clean = []
X_test_challenge_clean = []

a = 10 #len(X_train)
b = 10 #len(X_test_challenge)

def progress_bar(current, total, prefix='', suffix='', length=50):
    percent = ("{0:.1f}").format(100 * (current / float(total)))
    filled_length = int(length * current // total)
    bar = '█' * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end='\r')
    if current == total:
        print()

preprocessing_start_time = time.time()

for k in range(a):
    tokens = word_tokenize(header.normalize_accent(X_train[k].lower()), language='french')
    tokens = [word for word in tokens if word not in string.punctuation]
    X_train_clean.append(tokens)
    progress_bar(k + 1, a, prefix='X_train_raw_designation_clean :', suffix='Complete', length=50)

for k in range(b):
    tokens = word_tokenize(header.normalize_accent(X_test_challenge[k].lower()), language='french')
    tokens = [word for word in tokens if word not in string.punctuation]
    X_test_challenge_clean.append(tokens)
    progress_bar(k + 1, b, prefix='X_test_raw_designation_clean :', suffix='Complete', length=50)

X_train_strings = [' '.join(tokens) for tokens in X_train_clean]
X_test_strings = [' '.join(tokens) for tokens in X_test_challenge_clean]

preprocessing_end_time = time.time()
preprocessing_time_h, preprocessing_time_min, preprocessing_time_s = header.convert_seconds(
    preprocessing_end_time - preprocessing_start_time)

print(f"Preprocessed in {int(preprocessing_time_h)}h {int(preprocessing_time_min)}min {int(preprocessing_time_s)}s")


X_train_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete
X_test_raw_designation_clean : |██████████████████████████████████████████████████| 100.0% Complete
Preprocessed in 0h 0min 0s


[nltk_data] Downloading package punkt to /Users/welto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
X_train_strings

['olivia personalisiertes notizbuch 150 seiten punktraster ca din a5 rosen-design',
 "journal des arts le n 133 du 28/09/2001 l'art et son marche salon d'art asiatique a paris jacques barrere francois perrier la reforme des ventes aux encheres publiques le sna fete ses cent ans",
 'grand stylet ergonomique bleu gamepad nintendo wii u speedlink pilot style',
 'peluche donald europe disneyland 2000 marionnette a doigt',
 'la guerre des tuques',
 'afrique contemporaine n 212 hiver 2004 dossier japon afrique',
 'christof e bildungsprozessen auf der spur',
 'conquerant sept cahier couverture polypro 240 x 320 mm 96 pages 90g seyes incolore',
 'puzzle scooby-doo avec poster 2x35 pieces',
 'tente pliante v3s5-pro pvc blanc 3 x 4m50 longueur 4m50 largeur 3 m blanc h']